# Import libraries and data

In [1]:
## install finrl library
!pip install git+https://github.com/AI4Finance-LLC/FinRL-Library.git

  Cloning https://github.com/AI4Finance-LLC/FinRL-Library.git to /tmp/pip-req-build-3xz7m63p
  Running command git clone -q https://github.com/AI4Finance-LLC/FinRL-Library.git /tmp/pip-req-build-3xz7m63p
  Cloning https://github.com/quantopian/pyfolio.git to /tmp/pip-install-xndlpr4q/pyfolio_044ae4928be04ddb95a4ae3745ac9ea2
  Running command git clone -q https://github.com/quantopian/pyfolio.git /tmp/pip-install-xndlpr4q/pyfolio_044ae4928be04ddb95a4ae3745ac9ea2
  Cloning https://github.com/AI4Finance-Foundation/ElegantRL.git to /tmp/pip-install-xndlpr4q/elegantrl_d0159ca6828b4ef0a9ffd95e85589946
  Running command git clone -q https://github.com/AI4Finance-Foundation/ElegantRL.git /tmp/pip-install-xndlpr4q/elegantrl_d0159ca6828b4ef0a9ffd95e85589946


In [2]:
## import libraries
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('Agg')
%matplotlib inline
import datetime
import calendar

from finrl.apps import config
from finrl.neo_finrl.preprocessor.yahoodownloader import YahooDownloader
from finrl.neo_finrl.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl.neo_finrl.env_portfolio_allocation.env_portfolio import StockPortfolioEnv
from finrl.drl_agents.stablebaselines3.models import DRLAgent
from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline,convert_daily_return_to_pyfolio_ts


import sys
sys.path.append("../FinRL-Library")

import os
if not os.path.exists("./" + config.DATA_SAVE_DIR):
    os.makedirs("./" + config.DATA_SAVE_DIR)
if not os.path.exists("./" + config.TRAINED_MODEL_DIR):
    os.makedirs("./" + config.TRAINED_MODEL_DIR)
if not os.path.exists("./" + config.TENSORBOARD_LOG_DIR):
    os.makedirs("./" + config.TENSORBOARD_LOG_DIR)
if not os.path.exists("./" + config.RESULTS_DIR):
    os.makedirs("./" + config.RESULTS_DIR)

/usr/local/lib/python3.7/dist-packages/pyfolio/pos.py:27: UserWarning: Module "zipline.assets" not found; multipliers will not be applied to position notionals.
  'Module "zipline.assets" not found; multipliers will not be applied'


In [3]:
# URL to git repo
url = 'https://github.com/changjulian17/DataSciencePortfolio/blob/main/Investment_Portfolio/data/processed_data.pkl?raw=true'
df = pd.read_pickle(url)

# Design Environment
Considering the stochastic and interactive nature of the automated stock trading tasks, a financial task is modeled as a Markov Decision Process (MDP) problem. The training process involves observing stock price change, taking an action and reward's calculation to have the agent adjusting its strategy accordingly. By interacting with the environment, the trading agent will derive a trading strategy with the maximized rewards as time proceeds.

Our trading environments, based on OpenAI Gym framework, simulate live stock markets with real market data according to the principle of time-driven simulation.

## Training data split 2005-11-30 to 2020-01-01

In [4]:
train = data_split(df, '2005-11-30','2020-01-01')

In [5]:
train.shape

(24801, 18)

In [6]:
df.tail()

,date,open,high,low,close,volume,tic,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,cov_list,return_list
27736,2021-08-31,52.470001,52.520000,52.330002,52.410000,43226700.0,EEM,1.0,-0.276504,53.255365,49.519635,49.906189,78.207188,6.277385,51.614666,53.012122,"[[4.1802741524284105e-06, -1.8829709541513621e...",tic AGG COM EEM ... ...
27737,2021-08-31,0.000000,0.000000,0.000000,630.590000,0.0,GLD,0.0,0.496968,637.901073,600.379927,52.402475,83.613562,6.277385,622.136000,626.241500,"[[4.1802741524284105e-06, -1.8829709541513621e...",tic AGG COM EEM ... ...
27738,2021-08-31,117.330002,117.489998,117.120003,116.926025,7418400.0,IEF,1.0,0.076884,117.821486,116.121367,53.687664,-14.855131,7.496965,117.068840,116.272461,"[[4.1802741524284105e-06, -1.8829709541513621e...",tic AGG COM EEM ... ...
27739,2021-08-31,0.000000,0.000000,0.000000,190.572196,0.0,LOV,0.0,-0.400142,190.572203,190.572189,1.183841,-45.454545,7.496965,191.162715,192.190168,"[[4.1802741524284105e-06, -1.8829709541513621e...",tic AGG COM EEM ... ...
27740,2021-08-31,9448.330078,9451.559570,9419.410156,9433.580078,0.0,SNP,1.0,77.304140,9458.550010,9131.839053,62.965117,167.127295,21.356673,9250.471322,9087.068766,"[[4.1802741524284105e-06, -1.8829709541513621e...",tic AGG COM EEM ... ...


## Environment for Portfolio Allocation

In [7]:
#@title
import numpy as np
import pandas as pd
from gym.utils import seeding
import gym
from gym import spaces
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from stable_baselines3.common.vec_env import DummyVecEnv


class StockPortfolioEnv(gym.Env):
    """A single stock trading environment for OpenAI gym

    Attributes
    ----------
        df: DataFrame
            input data
        stock_dim : int
            number of unique stocks
        hmax : int
            maximum number of shares to trade
        initial_amount : int
            start money
        transaction_cost_pct: float
            transaction cost percentage per trade
        reward_scaling: float
            scaling factor for reward, good for training
        state_space: int
            the dimension of input features
        action_space: int
            equals stock dimension
        tech_indicator_list: list
            a list of technical indicator names
        turbulence_threshold: int
            a threshold to control risk aversion
        day: int
            an increment number to control date

    Methods
    -------
    _sell_stock()
        perform sell action based on the sign of the action
    _buy_stock()
        perform buy action based on the sign of the action
    step()
        at each step the agent will return actions, then 
        we will calculate the reward, and return the next observation.
    reset()
        reset the environment
    render()
        use render to return other functions
    save_asset_memory()
        return account value at each time step
    save_action_memory()
        return actions/positions at each time step
        

    """
    metadata = {'render.modes': ['human']}

    def __init__(self, 
                df,
                stock_dim,
                hmax,
                initial_amount,
                transaction_cost_pct,
                reward_scaling,
                state_space,
                action_space,
                tech_indicator_list,
                turbulence_threshold=None,
                lookback=252,
                day = 0):
        #super(StockEnv, self).__init__()
        #money = 10 , scope = 1
        self.day = day
        self.lookback=lookback
        self.df = df
        self.stock_dim = stock_dim
        self.hmax = hmax
        self.initial_amount = initial_amount
        self.transaction_cost_pct =transaction_cost_pct
        self.reward_scaling = reward_scaling
        self.state_space = state_space
        self.action_space = action_space
        self.tech_indicator_list = tech_indicator_list

        # action_space normalization and shape is self.stock_dim
        self.action_space = spaces.Box(low = 0, high = 1,shape = (self.action_space,)) 
        # Shape = (34, 30)
        # covariance matrix + technical indicators
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape = (self.state_space+len(self.tech_indicator_list),self.state_space))

        # load data from a pandas dataframe
        self.data = self.df.loc[self.day,:]
        self.covs = self.data['cov_list'].values[0]
        self.state =  np.append(np.array(self.covs), [self.data[tech].values.tolist() for tech in self.tech_indicator_list ], axis=0)
        self.terminal = False     
        self.turbulence_threshold = turbulence_threshold        
        # initalize state: inital portfolio return + individual stock return + individual weights
        self.portfolio_value = self.initial_amount

        # memorize portfolio value each step
        self.asset_memory = [self.initial_amount]
        # memorize portfolio return each step
        self.portfolio_return_memory = [0]
        self.actions_memory=[[1/self.stock_dim]*self.stock_dim]
        self.date_memory=[self.data.date.unique()[0]]

        
    def step(self, actions):
        # print(self.day)
        self.terminal = self.day >= len(self.df.index.unique())-1
        # print(actions)

        if self.terminal:
            df = pd.DataFrame(self.portfolio_return_memory)
            df.columns = ['daily_return']
            plt.plot(df.daily_return.cumsum(),'r')
            plt.savefig('results/cumulative_reward.png')
            plt.close()
            
            plt.plot(self.portfolio_return_memory,'r')
            plt.savefig('results/rewards.png')
            plt.close()

            print("=================================")
            print("begin_total_asset:{}".format(self.asset_memory[0]))           
            print("end_total_asset:{}".format(self.portfolio_value))

            df_daily_return = pd.DataFrame(self.portfolio_return_memory)
            df_daily_return.columns = ['daily_return']
            if df_daily_return['daily_return'].std() !=0:
              sharpe = (252**0.5)*df_daily_return['daily_return'].mean()/ \
                       df_daily_return['daily_return'].std()
              print("Sharpe: ",sharpe)
            print("=================================")
            
            return self.state, self.reward, self.terminal,{}

        else:
            #print("Model actions: ",actions)
            # actions are the portfolio weight
            # normalize to sum of 1
            #if (np.array(actions) - np.array(actions).min()).sum() != 0:
            #  norm_actions = (np.array(actions) - np.array(actions).min()) / (np.array(actions) - np.array(actions).min()).sum()
            #else:
            #  norm_actions = actions
            weights = self.softmax_normalization(actions) 
            #print("Normalized actions: ", weights)
            self.actions_memory.append(weights)
            last_day_memory = self.data

            #load next state
            self.day += 1
            self.data = self.df.loc[self.day,:]
            self.covs = self.data['cov_list'].values[0]
            self.state =  np.append(np.array(self.covs), [self.data[tech].values.tolist() for tech in self.tech_indicator_list ], axis=0)
            #print(self.state)
            # calcualte portfolio return
            # individual stocks' return * weight
            portfolio_return = sum(((self.data.close.values / last_day_memory.close.values)-1)*weights)
            # update portfolio value
            new_portfolio_value = self.portfolio_value*(1+portfolio_return)
            self.portfolio_value = new_portfolio_value

            # save into memory
            self.portfolio_return_memory.append(portfolio_return)
            self.date_memory.append(self.data.date.unique()[0])            
            self.asset_memory.append(new_portfolio_value)

            # the reward is the new portfolio value or end portfolo value
            self.reward = new_portfolio_value 
            #print("Step reward: ", self.reward)
            #self.reward = self.reward*self.reward_scaling

        return self.state, self.reward, self.terminal, {}

    def reset(self):
        self.asset_memory = [self.initial_amount]
        self.day = 0
        self.data = self.df.loc[self.day,:]
        # load states
        self.covs = self.data['cov_list'].values[0]
        self.state =  np.append(np.array(self.covs), [self.data[tech].values.tolist() for tech in self.tech_indicator_list ], axis=0)
        self.portfolio_value = self.initial_amount
        #self.cost = 0
        #self.trades = 0
        self.terminal = False 
        self.portfolio_return_memory = [0]
        self.actions_memory=[[1/self.stock_dim]*self.stock_dim]
        self.date_memory=[self.data.date.unique()[0]] 
        return self.state
    
    def render(self, mode='human'):
        return self.state
        
    def softmax_normalization(self, actions):
        numerator = np.exp(actions)
        denominator = np.sum(np.exp(actions))
        softmax_output = numerator/denominator
        return softmax_output

    
    def save_asset_memory(self):
        date_list = self.date_memory
        portfolio_return = self.portfolio_return_memory
        #print(len(date_list))
        #print(len(asset_list))
        df_account_value = pd.DataFrame({'date':date_list,'daily_return':portfolio_return})
        return df_account_value

    def save_action_memory(self):
        # date and close price length must match actions length
        date_list = self.date_memory
        df_date = pd.DataFrame(date_list)
        df_date.columns = ['date']
        
        action_list = self.actions_memory
        df_actions = pd.DataFrame(action_list)
        df_actions.columns = self.data.tic.values
        df_actions.index = df_date.date
        #df_actions = pd.DataFrame({'date':date_list,'actions':action_list})
        return df_actions

    def _seed(self, seed=None):
        self.np_random, seed = seeding.np_random(seed)
        return [seed]

    def get_sb_env(self):
        e = DummyVecEnv([lambda: self])
        obs = e.reset()
        return e, obs

In [8]:
stock_dimension = len(train.tic.unique())
state_space = stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

Stock Dimension: 7, State Space: 7


In [9]:
env_kwargs = {
    "hmax": 100, 
    "initial_amount": 1000000, 
    "transaction_cost_pct": 0.001, 
    "state_space": state_space, 
    "stock_dim": stock_dimension, 
    "tech_indicator_list": config.TECHNICAL_INDICATORS_LIST, 
    "action_space": stock_dimension, 
    "reward_scaling": 1e-4
    
}

e_train_gym = StockPortfolioEnv(df = train, **env_kwargs)

In [10]:
env_train, _ = e_train_gym.get_sb_env()
print(type(env_train))

<class 'stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv'>


# Model PPO

In [11]:
agent = DRLAgent(env = env_train)
PPO_PARAMS = {
    "n_steps": 2048,
    "ent_coef": 0.005,
    "learning_rate": 0.0001,
    "batch_size": 128,
}
model_ppo = agent.get_model("ppo",model_kwargs = PPO_PARAMS)

{'n_steps': 2048, 'ent_coef': 0.005, 'learning_rate': 0.0001, 'batch_size': 128}
Using cpu device


In [12]:
trained_ppo = agent.train_model(model=model_ppo, 
                             tb_log_name='ppo',
                             total_timesteps=1280000)

Streaming output truncated to the last 5000 lines.
|    entropy_loss         | -9.93     |
|    explained_variance   | 0         |
|    learning_rate        | 0.0001    |
|    loss                 | 5.28e+14  |
|    n_updates            | 4060      |
|    policy_gradient_loss | -2.55e-07 |
|    reward               | 1353987.1 |
|    std                  | 1         |
|    value_loss           | 1.04e+15  |
---------------------------------------
---------------------------------------
| time/                   |           |
|    fps                  | 426       |
|    iterations           | 408       |
|    time_elapsed         | 1959      |
|    total_timesteps      | 835584    |
| train/                  |           |
|    approx_kl            | 0.0       |
|    clip_fraction        | 0         |
|    clip_range           | 0.2       |
|    entropy_loss         | -9.93     |
|    explained_variance   | -3.58e-07 |
|    learning_rate        | 0.0001    |
|    loss                 | 4

## State, Action, Reward - Sample

In [ ]:
## Sample a state
data = train.loc[5,:]
covs = data['cov_list'].values[0] # only need cov matrix for all 7 

# state is the cov matrix of each asset and technical indicators there after
state =  np.append(np.array(covs), [data[tech].values.tolist() for tech in config.TECHNICAL_INDICATORS_LIST], axis=0)
col_list = list(data.tic) + list(config.TECHNICAL_INDICATORS_LIST)
pd.DataFrame(state.T,index=data.tic, columns = col_list)

In [ ]:
## Sample an action
# action is the portfolio allocation on each day
e_train_gym.save_action_memory()

In [ ]:
## Sample a reward
# reward in this case is the total asset value
e_train_gym.asset_memory[-5:]

## Trading
Assume that we have $1,000,000 initial capital at 2020-01-01. We use the PPO model to trade the 5 assets.

In [ ]:
trade = data_split(df,'2020-01-01', '2021-8-31')
e_trade_gym = StockPortfolioEnv(df = trade, **env_kwargs)

In [ ]:
trade.shape

In [ ]:
df_daily_return, df_actions = DRLAgent.DRL_prediction(model=trained_ppo,
                        environment = e_trade_gym)

In [ ]:
df_daily_return

In [ ]:
df_daily_return.to_csv('df_daily_return.csv')

# Backtest Our Strategy
Backtesting plays a key role in evaluating the performance of a trading strategy. Automated backtesting tool is preferred because it reduces the human error. We usually use the Quantopian pyfolio package to backtest our trading strategies. It is easy to use and consists of various individual plots that provide a comprehensive image of the performance of a trading strategy.

In [ ]:
from pyfolio import timeseries
DRL_strat = convert_daily_return_to_pyfolio_ts(df_daily_return)
perf_func = timeseries.perf_stats 
perf_stats_all = perf_func( returns=DRL_strat, 
                              factor_returns=DRL_strat, 
                                positions=None, transactions=None, turnover_denom="AGB")

In [ ]:
print("==============PPO Strategy Stats===========")
perf_stats_all

---
need to create baseline with balanced dragon portfolio

In [ ]:
baseline_df = data_split(df, start=df_daily_return.loc[0,'date'], end='2021-8-31')
drg_wt = [.1,.2,.1,.2,.1,.2,.1]

In [ ]:
## CREATE DRAGON PORTFOLIO IN ABOVE FORMAT 
# start from DF
def get_drg_baseline(dataframe):
  # select relevant columns
  data = dataframe[dataframe.columns[:8]]
  data = data.groupby(['date','tic']).mean()
  # prepare empty dictionary
  drg_dict = {'date':[], 'open':[], 'high':[], 'low':[], 'close':[],\
            'volume':[], 'tic':'DRG' ,'day':[]}
  # prepare aggregated features
  keys = ['open', 'high', 'low', 'close', 'volume']

  # for each day attach the weighted sum and the date
  for date in dataframe.date.unique():
    # filter for date
    temp = data.loc[date]  
    values = temp[temp.columns[:-1]].mul(drg_wt,axis=0).sum()
    day = temp[temp.columns[-1]][0]

    # append lists
    drg_dict['date'].append(date)
    drg_dict['day'].append(day)
    for key, value in zip(keys, values):
      drg_dict[key].append(value)

  # create drg df
  dataframe = pd.DataFrame(drg_dict)

  return dataframe

In [ ]:
#baseline stats
print("==============Get Baseline Stats===========")
baseline_df = get_drg_baseline(baseline_df)

stats = backtest_stats(baseline_df, value_col_name = 'close')


## BackTestPlot

In [ ]:
import pyfolio
%matplotlib inline

"""baseline_df = get_baseline(
        ticker='^SP500TR', 
        start = df_daily_return.loc[0,'date'],
        end = df_daily_return.loc[len(df_daily_return)-1,'date'])"""

baseline_returns = get_daily_return(baseline_df, value_col_name="close")

with pyfolio.plotting.plotting_context(font_scale=1.1):
        pyfolio.create_full_tear_sheet(returns = DRL_strat,
                                       benchmark_rets=baseline_returns, set_context=False)